# Fun Visualisations of the Tracked Events using `plotX`

In [1]:
from getpass import getuser
from pathlib import Path

import dask
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import marEx
import marEx.helper as hpc

In [ ]:
# Directories
scratch_dir = Path("/scratch") / getuser()[0] / getuser()  # Lustre Scratch Directory

fpath_ckdtree = scratch_dir / "grid_files" / "ckdtree" / "rectgrids"
fpath_tgrid = scratch_dir / "grid_files" / "fpath_tgrid.zarr/"
file_name = scratch_dir / "mhws" / "extreme_events_merged_unstruct.zarr"
plot_dir = scratch_dir / "mhws" / "plots"

In [2]:
# Start Dask Cluster
client = hpc.start_local_cluster(
    n_workers=32, threads_per_worker=1, scratch_dir=scratch_dir / "clients"
)  # Specify temporary scratch directory for dask to use

In [4]:
# Define the Structure of the Unstructured Grid (either the triangulation file or the cKDTree for the grid)

marEx.specify_grid(grid_type="unstructured", fpath_ckdtree=fpath_ckdtree)

In [5]:
# Import Tracked Extremes DataSet
extreme_events_ds = xr.open_zarr(str(file_name), chunks={})
extreme_events_ds

In [6]:
# Extract the Events ID Field
ID_field = extreme_events_ds.ID_field
ID_field

## Make a Movie using `Xplot.animate`
N.B.: Making `dask`-powered movies using `animate()` cannot be threaded !

In [ ]:
# Choose a subset of the ID field
ID_field_subset = ID_field.sel(time=slice("2025-01-01", "2027-01-01"))

# Setup Plotting Configuration & Animate
config = marEx.PlotConfig(plot_IDs=True)
ID_field_subset.Xplot.animate(config, plot_dir=plot_dir, file_name="movie_ID_field_unstruct")

In [ ]:
%%HTML
<video width="640" height="480" controls>
    <source src="../../media/movie_ID_field_unstruct.mp4" type="video/mp4">
    Your browser does not support the video tag.
</video>

cf. Centroid-based partitioning:

In [ ]:
# Choose a subset of the ID field created with Centroid Partitioning
ID_field_centroid_subset = xr.open_zarr(str(file_name).replace("merged", "merged_centroid"), chunks={}).ID_field.sel(
    time=slice("2025-01-01", "2027-01-01")
)

# Setup Plotting Configuration & Animate
config = marEx.PlotConfig(plot_IDs=True)
ID_field_centroid_subset.plotX.animate(config, plot_dir=plot_dir, file_name="movie_ID_field_centroid_unstruct")

## Plot Consecutive Days of Extreme Events

In [ ]:
# Choose a subset of the ID field
ID_field_subset = ID_field.sel(time=slice("2025-01-01", "2025-01-06"))

# Setup Plotting Configuration
config = marEx.PlotConfig(plot_IDs=True)
fig, ax = ID_field_subset.Xplot.multi_plot(config, col="time", col_wrap=3);

## Plot Consecutive Months of Extreme Events

In [ ]:
# Choose the first day of each month from a subset of the ID field
ID_field_subset_day1 = ID_field.sel(time=slice("2025-01-01", "2025-12-31")).resample(time="MS").first()

# Setup Plotting Configuration
config = marEx.PlotConfig(plot_IDs=True)
fig, ax = ID_field_subset_day1.Xplot.multi_plot(config, col="time", col_wrap=3);

## Plot Global Extreme Event Frequency

In [ ]:
# Calculate the frequency of Event Occurrence
event_frequency = (ID_field > 0).astype(float).mean("time")

# Setup Plotting Configuration
config = marEx.PlotConfig(var_units="MHW Frequency", cmap="hot_r", cperc=[0, 96], grid_labels=True)
fig, ax, im = event_frequency.Xplot.single_plot(config)

## Plot Seasonal Extreme Event Frequency

In [ ]:
# Calculate the frequency of Event Occurrence per season
seasonal_frequency = (ID_field > 0).groupby("time.season").mean(dim="time")

# Setup Plotting Configuration
config = marEx.PlotConfig(var_units="MHW Frequency", cmap="hot_r", cperc=[0, 96], grid_labels=True)
fig, ax = seasonal_frequency.plotX.multi_plot(config, col="season", col_wrap=2);

## Find & Plot a Few of the Longest Events

In [ ]:
events_duration = extreme_events_ds.time_end - extreme_events_ds.time_start
longest_events = events_duration.sortby(events_duration, ascending=False).ID

for ID in longest_events[:10].values:
    print(
        f"ID: {ID:<6}   Start Day: {extreme_events_ds.time_start.sel(ID=ID).dt.strftime('%Y-%m-%d').values}  -->  Duration: {events_duration.sel(ID=ID).dt.days.values:<4} days"
    )

In [ ]:
# Calculate the local duration of the longest events
long_events = ID_field == (longest_events[:9]).chunk({"ID": 1})
long_events_local_duration = (long_events > 0).sum("time")

# Setup Plotting Configuration
config = marEx.PlotConfig(var_units="Duration (days)", cmap="hot_r", cperc=[0, 100])
fig, ax = long_events_local_duration.Xplot.multi_plot(config, col="ID", col_wrap=3);

## ID Maximum and Mean Event Area

In [ ]:
areas_max = extreme_events_ds.area.max().compute() / (1.0e3**2)
print(f"Maximum object area: {areas_max.values:.0f} km²")

areas_mean = extreme_events_ds.area.mean().compute() / (1.0e3**2)
print(f"Mean object area: {areas_mean.values:.0f} km²")

## Time-Series Plot of Events Area

In [ ]:
area_mean = extreme_events_ds.area.mean("ID").resample(time="ME").mean() / (1.0e3**2)

area_10 = extreme_events_ds.area.reduce(np.nanpercentile, q=10, dim="ID").resample(time="ME").mean() / (1.0e3**2)
area_90 = extreme_events_ds.area.reduce(np.nanpercentile, q=90, dim="ID").resample(time="ME").mean() / (1.0e3**2)

plt.figure(figsize=(15, 6))
area_mean.plot(label="Mean Area", color="k", lw=2)
plt.fill_between(area_mean.time.values, area_10, area_90, alpha=0.5)
plt.ylim([0, 8e7])
plt.ylabel(r"Event Area [km$^2$]");

## Hovmüller Plot of MHW Monthly Frequency vs Latitude
We need to first bin the unstructured data in latitude.

In [ ]:
# Group by latitude bins and calculate mean within each bin
lat_bins = np.linspace(-90.0, 90.0, 360 + 1)
binned_presence = (ID_field > 0).groupby_bins(ID_field.lat, bins=lat_bins).mean(dim="ncells")

# Calculate bin centers for better labelling
bin_centers = [(lat_bins[i] + lat_bins[i + 1]) / 2 for i in range(len(lat_bins) - 1)]
binned_presence = binned_presence.assign_coords(lat_center=("lat_bins", bin_centers))

# Resample in time to get monthly means
spatial_presence = binned_presence.resample(time="ME").mean()

fig, ax = plt.subplots(figsize=(15, 6))
im = spatial_presence.plot(
    ax=ax,
    cmap="hot",
    x="time",
    y="lat_center",
    cbar_kwargs={"label": "MHW Presence Frequency", "extend": "both"},
)

ax.set_xlabel("Time")
ax.set_ylabel("Latitude")
ax.grid(True, linestyle="--", alpha=0.6)

## Seasonal Cycle of MHW vs Latitude
We again need to bin the unstructured data in latitude. (Reuse the `binned_presence` from above.)

In [ ]:
spatial_presence_seasonal = binned_presence.groupby("time.dayofyear").mean()

fig, ax = plt.subplots(figsize=(15, 6))
im = spatial_presence_seasonal.plot(
    ax=ax,
    cmap="hot",
    x="dayofyear",
    y="lat_center",
    cbar_kwargs={"label": "MHW Seasonal Frequency", "extend": "both"},
)

months = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]
days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
month_positions = np.cumsum([0] + days_per_month[:-1]) + 15

ax.set_xticks(month_positions)
ax.set_xticklabels(months)
ax.set_xlabel("Month")
ax.set_ylabel("Latitude")
ax.grid(True, linestyle="--", alpha=0.6)